In [ ]:
# Standard library
from collections import OrderedDict
import os

# Third-party
import astropy.time as atime
from astropy import log as logger
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Project
from thejoker import Paths
paths = Paths()
from thejoker.data import RVData
from thejoker.units import usys
from thejoker.util import quantity_from_hdf5
from thejoker.celestialmechanics import OrbitalParams, SimulatedRVOrbit, rv_from_elements
from thejoker.plot import plot_rv_curves, plot_corner

plt.style.use('../thejoker/thejoker.mplstyle')

In [ ]:
data_filename = "../data/experiment2.h5"

plot_path = "../paper/figures/"
# plot_path = "../plots/Experiment2"
os.makedirs(plot_path, exist_ok=True)

## Read the data

In [ ]:
all_data = OrderedDict()
with h5py.File(data_filename, 'r') as f:
    apogee_id = f.attrs['APOGEE_ID']
    for key in sorted([int(x) for x in f.keys()]):
        g = f[str(key)]

        bmjd = g['mjd'][:]
        rv = quantity_from_hdf5(g, 'rv')
        rv_err = quantity_from_hdf5(g, 'rv_err')
        
        all_data[key] = RVData(bmjd, rv, stddev=rv_err)

In [ ]:
fig,axes = plt.subplots(8, 1, figsize=(5,10), 
                        sharex=True, sharey=True)   

for i,key in enumerate(list(all_data.keys())):
    data = all_data[key]
        
    data.plot(ax=axes[i], rv_unit=u.km/u.s, markersize=3, capsize=0,
              elinewidth=0, color='k', zorder=1000, alpha=0.75)
        
    if key > 0:
        not_included = np.array([x not in data._rv for x in all_data[0]._rv])
        reject_data = all_data[0][not_included]

        axes[i].plot(reject_data.t.tcb.mjd, reject_data.rv.to(u.km/u.s).value, 
                     marker='x', zorder=100, linestyle='none',
                     markersize=4, markeredgewidth=1, color='#de2d26')
        
fig.tight_layout()

### Read in Troup's fit values for the orbital parameters:

In [ ]:
all_troup_data = np.genfromtxt(os.path.join(paths.root, "data", "troup16-dr12.csv"),
                               delimiter=",", names=True, dtype=None)
troup_data = all_troup_data[all_troup_data['APOGEE_ID'].astype(str) == apogee_id]

In [ ]:
troup_data['PERIOD'], troup_data['ECC']

In [ ]:
dmjd = all_data[0].t.tcb.mjd.max() - all_data[0].t.tcb.mjd.min()
t_grid = np.linspace(all_data[0].t.tcb.mjd.min() -0.25*dmjd, 
                     all_data[0].t.tcb.mjd.max() + 0.25*dmjd, 
                     1024)

rv_unit = u.km/u.s

In [ ]:
n_panel_per_fig = 4

for j in range(len(all_data) // n_panel_per_fig):
    fig,axes = plt.subplots(n_panel_per_fig, 1, figsize=(6.5,10), 
                            sharex=True, sharey=True)   

    for i,key in enumerate(list(all_data.keys())[j*n_panel_per_fig:(j+1)*n_panel_per_fig]):
        data = all_data[key]
        
        # read the orbital parameters
        samples_filename = "../cache/experiment2-{}.h5".format(key)
        with h5py.File(samples_filename, 'r') as g:
            opars = OrbitalParams.from_hdf5(g)
        
        plot_rv_curves(opars[:512], t_grid, t_offset=data.t_offset, rv_unit=rv_unit, 
                       ax=axes[i], plot_kwargs={'color': '#888888', 'zorder': -100},
                       add_labels=False)
        axes[i].set_rasterization_zorder(-1)
        
        data.plot(ax=axes[i], rv_unit=u.km/u.s, markersize=3, capsize=0,
                  elinewidth=0, color='k', zorder=1000, alpha=0.75)
        
        if key > 0:
            not_included = np.array([x not in data._rv for x in all_data[0]._rv])
            reject_data = all_data[0][not_included]
            
            axes[i].plot(reject_data.t.tcb.mjd, reject_data.rv.to(u.km/u.s).value, 
                         marker='x', zorder=100, linestyle='none',
                         markersize=4, markeredgewidth=1, color='#de2d26')
        
        axes[i].text(55650, 15, "$N={}$, M={}".format(len(data), len(opars)), va='top', fontsize=18)
    
    axes[0].set_ylabel('RV [km s$^{-1}$]')
    axes[-1].set_xlabel('BMJD')
    
    axes[0].set_xlim(t_grid.min(), t_grid.max())
    axes[0].set_ylim(-40, 20)

    fig.tight_layout()
    
    # change to: dpi=256 for production?
    fig.savefig(os.path.join(plot_path, 'exp2-rv-curves-{}.pdf'.format(j)), dpi=100) 